In [88]:
# !pip install selenium fpdf

# !pip install selenium==3.141.0 
# !pip install --upgrade urllib3==1.26.16
# !pip install selenium webdriver_manager reportlab
!pip install undetected-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 42.3 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 733.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47048 sha256=ea6238953878824e7592fe5d5784212ed186a70caaee92a348a15e1aeecffafb
  Stored in directory: /Users/khair1212/Library/Caches/pip/wheels/d1/9c/b4/ccdca75ca356899a9b65190f048a03058f1fe23602c642b88e
Successfully built undetected-chromedriver
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 0.58.0
    Uninstalling websocket-client-0.58.0:
      Successfully uninstalled websocket-client-0.58.0
  Attempting uninstall: selenium
    Found existing installation: selenium 3.141.0
    Uninstalling selenium-3.141.0:
      Successfully uninstalled seleni

In [151]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
# from fpdf import FPDF
from selenium.webdriver.support import expected_conditions as EC
import os 
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import time
from reportlab.lib import colors
from reportlab.lib.utils import ImageReader
from reportlab.platypus import Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

from reportlab.lib.enums import TA_JUSTIFY
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, urljoin
import textwrap
import requests
import undetected_chromedriver as uc
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException


In [2]:
os.path.exists('chromedriver')

True

In [164]:
options = selenium.webdriver.chrome.options.Options()
options.add_argument("--incognito")
driver = webdriver.Chrome(options=options)
subdriver = webdriver.Chrome(options=options) 

In [131]:
from selenium.webdriver.chrome.service import Service

# driver = webdriver.Chrome()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--use_subprocess")

browser = webdriver.Chrome(options=chrome_options)
visited_urls = set() 

In [91]:
base_url = 'https://www.brainstation-23.com/'
# sections = driver.find_element('xpath', '/html/body/div/div[1]/div/main/div')

driver.get(base_url)
try:
    sections = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'main-navigation'))
    )
    print("Element found")
except TimeoutException:
    print("Element not found after waiting")
except NoSuchElementException:
    print("No such element")



Element found


In [128]:
base_url = 'https://brainstation-23.com/'

# Navigation Content Extraction

In [110]:
sections = {}
driver.get(base_url)

try:
    main_navigation = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'main-navigation')) 
    )  
    # main_navigation = driver.find_element(By.CLASS_NAME, 'main-navigation')
    links = main_navigation.find_elements(By.TAG_NAME, 'a') 
    for link in links: 
        section_name = link.text.strip() 
        section_url = link.get_attribute('href') 
        if section_name and section_url: 
            try: 
                # Use requests to get the redirected URL
                response = requests.head(section_url, allow_redirects=True) 
                final_url = response.url 
                sections[section_name] = final_url 
            except Exception as e: 
                print(f"Error occurred while fetching redirected URL for {section_name}: {e}") 
except Exception as e:
    print(f"Error occurred while extracting sections: {e}") 


In [167]:
sections 


{'Home': 'https://brainstation-23.com/?1',
 'Services': 'https://brainstation-23.com/services/',
 'Industries': 'https://brainstation-23.com/industries/',
 'Case Studies': 'https://brainstation-23.com/case-studies/',
 'Blog': 'https://brainstation-23.com/blog/',
 'About Us': 'https://brainstation-23.com/about/',
 'Investor Relations': 'https://brainstation-23.com/investor-relations/',
 'Contact Us': 'https://brainstation-23.com/contact/'}

In [127]:
home = {'Home': sections['Home']} 


## Define Retry_click function to escape StaleElementReferenceException 

In [31]:
def retrying_find_click(by, attempts=2):
    result = False
    for attempt in range(attempts):
        try:
            driver.find_element(by).click() 
            result = True   
            break   
        except Exception as e:
            if isinstance(e, StaleElementReferenceException): 
                time.sleep(1)  
            else:
                raise e 
    return result 

In [84]:
visited_urls = set()

In [165]:
def fetch_section(section_name, start_url):
    visited_urls.clear()  
    fetch_page(section_name, start_url)


def fetch_page(section_name, start_url):
    
    urls_to_visit = [(start_url, section_name)]
    
    while urls_to_visit:
        current_url, current_section = urls_to_visit.pop(0)
        if current_url in visited_urls:
            continue
        visited_urls.add(current_url)

        # try:
        #     # Use requests to handle redirections 
        #     response = requests.head(current_url, allow_redirects=True) 
        #     final_url = response.url 
        # except Exception as e: 
        #     print(f"Error occurred while handling redirection for {current_url}: {e}") 
        #     continue 
        
        print(current_url)
        driver.get(current_url)
        
        # Expanding all expandable items 
        try:
            while True:
                read_more_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'read more')] | //a[contains(text(), 'read more')]")
                if not read_more_buttons:
                    break
                for button in read_more_buttons:
                    retrying_find_click((By.XPATH, "//button[contains(text(), 'read more')]"))
                    retrying_find_click((By.XPATH, "//a[contains(text(), 'read more')]"))
                    time.sleep(1)  
        except:
            pass
        
        # Get the text content of the page
        try:
            # remove scripts of the page content
            driver.execute_script("var elements = document.getElementsByTagName('script'); while(elements[0]) { elements[0].parentNode.removeChild(elements[0]); }")
            
            # Get the body element
            body_element = driver.find_element(By.TAG_NAME, 'body')
            
            # Remove navigation elements
            navigation_elements = driver.find_elements(By.XPATH, "//header | //footer | //nav | //div[contains(@class, 'main-navigation')] | //div[contains(@class, 'navbar')]")
            for element in navigation_elements:
                try:
                    driver.execute_script("arguments[0].remove()", element)
                except:
                    continue
            
            # Re-fetch the body element after removing navigation elements to avoid stale element reference
            body_element = driver.find_element(By.TAG_NAME, 'body')
            
            
            page_content = body_element.text
            
            with open(f'{current_section}_content.txt', 'a', encoding='utf-8') as file:
                file.write(f"URL: {current_url}\n\n")
                file.write(page_content)
                file.write("\n" + "="*80 + "\n")
                
        except Exception as e:
            print(f"An error occurred while extracting text: {e}")


        links = driver.find_elements(By.TAG_NAME, 'a')
        
        for link in links: 
            try:
                href = link.get_attribute('href') 
            except StaleElementReferenceException:
                # Refresh the list of links and try again
                links = driver.find_elements(By.TAG_NAME, 'a') 
                href = None 
                for refreshed_link in links: 
                    if refreshed_link == link: 
                        href = refreshed_link.get_attribute('href') 
                        break 
            if href: 
                parsed_href = urlparse(href) 
               
                if parsed_href.scheme in ['http', 'https'] and parsed_href.netloc == urlparse("https://brainstation-23.com").netloc: 
                    absolute_url = urljoin(base_url, parsed_href.path) 
                    if absolute_url not in home.items(): 
                        subdriver.get(absolute_url) 
                        absolute_url = subdriver.current_url 
                        print("absolute_url:", absolute_url) 
                        if absolute_url not in visited_urls: 
                            print("Not in visited URL") 
                            urls_to_visit.append((absolute_url, current_section)) 
                            


In [168]:

for section, url in sections.items():
    fetch_section(section, url)


https://brainstation-23.com/?1
absolute_url: https://brainstation-23.com/?1
absolute_url: https://brainstation-23.com/contact/
Not in visited URL
absolute_url: https://brainstation-23.com/ui-ux-tips-techniques/
Not in visited URL
absolute_url: https://brainstation-23.com/ui-ux-tips-techniques/
Not in visited URL
absolute_url: https://brainstation-23.com/digital-banking-services/
Not in visited URL
absolute_url: https://brainstation-23.com/digital-banking-services/
Not in visited URL
absolute_url: https://brainstation-23.com/optimize-hr-processes-with-the-right-hrms-solution/
Not in visited URL
absolute_url: https://brainstation-23.com/optimize-hr-processes-with-the-right-hrms-solution/
Not in visited URL
absolute_url: https://brainstation-23.com/transforming-the-fashion-experience/
Not in visited URL
absolute_url: https://brainstation-23.com/transforming-the-fashion-experience/
Not in visited URL
absolute_url: https://brainstation-23.com/resource-augmentation-in-21st-century/
Not in vi

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.127)


In [ ]:
# Sequentially fetch each section based on the navigation items
for section, url in home.items():
    fetch_section(section, url)


In [25]:
def get_history(driver):
    extracted_histories = driver.find_element('xpath','//*[@id="post-7816"]/div/section[5]/div/div/div/div[3]/div/div/div/div/div').text
    segments = extracted_histories.strip().split('\n')

    year_start_index = None
    for i, part in enumerate(segments):
        if part.strip().isdigit() or '-' in part.strip():
            year_start_index = i
            break
    titles_descriptions = segments[:year_start_index]
    years = segments[year_start_index:]

    result = {}
    for i in range(0, len(titles_descriptions), 2):
        title = titles_descriptions[i].strip()
        description = titles_descriptions[i+1].strip()
        year = years[i//2].strip()  # integer division to get corresponding year
        result[title] = {'year': year, 'description': description}

    return result 
    

In [26]:
def scrape_about(driver):
    driver.get('https://www.nikles.com/about/')
    data = {}
    intro = driver.find_element('xpath', '//*[@id="post-7816"]/div/section[3]/div/div[1]/div/div[3]/div').text
    history = get_history(driver)
    philosophy = driver.find_element('xpath','//*[@id="post-7816"]/div/section[7]/div/div[1]/div').text
    data['Nikles in short']  = intro
    data['History'] = history
    data['Philosophy'] = philosophy
    return data 

In [40]:
driver.get('https://brainstation-23.com/about/')
data = {}
intro = driver.find_element('xpath', '//*[@id="page"]').text

# history = get_history(driver)
# philosophy = driver.find_element('xpath','//*[@id="post-7816"]/div/section[7]/div/div[1]/div').text
# data['Nikles in short']  = intro
# data['History'] = history
# data['Philosophy'] = philosophy


In [42]:
print(intro)

Home
Services
Industries
Case Studies
Blog
About Us
Investor Relations
Contact Us
Our Company
Our Story
It was in 2006, with little capital but a pocketful of belief our CEO, Raisul Kabir started Brain Station 23, a software company, right after graduating from BUET. The new company initially focused on the international market with the local market added in 2010. Since then the company has shown a continuous growth and currently employs over 700+ software engineers. Brain Station 23 is now not only an established name in Bangladesh but also in countries like the USA, UK, Netherlands, Denmark, Japan, Norway, Sweden, Germany, Canada, Switzerland, Turkey and the Middle East etc.
Read More
Our Mission
Your trusted companion for digital leadership by empowering people to achieve more with less
Our Vision
To be the fastest digital transformation and innovation partner by engaging global talents thus creating positive impact.
Leadership Values
Ownership - Take Responsibility And Own The Chal

In [27]:
def scrape_technologies(driver):
    driver.get('https://www.nikles.com/technologies/')
    section = driver.find_element('xpath', '//*[@id="post-7864"]/div/section[2]')
    data = {}
    h3_elements = section.find_elements(By.TAG_NAME, 'h3')
    p_elements = section.find_elements(By.TAG_NAME, 'p')

    h3_texts = [element.text for element in h3_elements]
    p_texts = [element.text for element in p_elements]
    
    data = {title: desc for title, desc in zip(h3_texts, p_texts)}
    
    return data

In [28]:
def scrape_luxury_finishes(driver):
    driver.get('https://www.nikles.com/nikles-luxury-finishes/')
    data = {}
    info = driver.find_element('xpath','//*[@id="content"]/div/section[2]/div/div/div/section/div/div[1]/div/div/div/h1').text
    tech = driver.find_element('xpath', '//*[@id="content"]/div/section[4]/div/div/div/div[1]/div/h1').text
    polished_item_containers = driver.find_elements(By.CLASS_NAME, 'elementor-widget-container')
    polished_items = {}
    for container in polished_item_containers:
        h1_elements = container.find_elements(By.TAG_NAME, 'h1')
        p_elements = container.find_elements(By.TAG_NAME, 'p')

        if h1_elements and p_elements:
            for h1, p in zip(h1_elements, p_elements):
                polished_items[h1.text] = p.text

    polished_items
    
    data = {'Info': info, 'Technology': tech, 'Polished Items': polished_items}
    return data 

In [29]:
def scrape_news(driver):
    driver.get('https://www.nikles.com/news/')
    read_more_links = driver.find_elements(By.CLASS_NAME, "elementor-post__read-more")
    data = {}
    for link in read_more_links:
        try: 
            href = link.get_attribute('href')
            driver.execute_script("window.open(arguments[0]);", href)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)

            title = driver.find_element(By.TAG_NAME, 'h1').text
            author = driver.find_element(By.CLASS_NAME, 'byline').find_element(By.TAG_NAME, 'a').text
            category = driver.find_element(By.CLASS_NAME, 'cat-links').find_element(By.TAG_NAME, 'a').text
            posted_time = driver.find_element(By.CLASS_NAME, 'posted-on').find_element(By.TAG_NAME, 'time').text
            content = driver.find_element(By.CLASS_NAME, 'entry-content').find_element(By.TAG_NAME, 'p').text
            data[title] = {'author': author, 'category': category, 'posted_time': posted_time, 'content': content}
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except Exception as e:
            print(f"An error occurred: {e}")
            break
    return data

In [30]:
def scrape_all():
    about_data = scrape_about(driver)
    technologies_data = scrape_technologies(driver)
    luxury_finishes_data = scrape_luxury_finishes(driver)
    news_data = scrape_news(driver)
    
    return {
        'About': about_data,
        'Technologies': technologies_data,
        'Luxury Finishes': luxury_finishes_data,
        'News': news_data
    }

In [14]:
def draw_text_block(c, title, text, y, font_bold_size=12, font_size=10):
    c.setFont("Helvetica-Bold", font_bold_size)
    c.drawString(left_margin, y, title)
    y -= 30
    c.setFont("Helvetica", font_size)
    
    # Calculate the width of the text block
    text_block_width = page_width - left_margin - right_margin
    
    for line in text.split('\n'):
        wrapped_lines = wrap_text(line, text_block_width)
        for wrapped_line in wrapped_lines:
            c.drawString(left_margin, y, wrapped_line)
            y -= 14
            if y < bottom_margin:
                c.showPage()
                y = page_height - top_margin
                c.setFont("Helvetica", font_size)
    return y - 20

def wrap_text(text, max_width):
    return simpleSplit(text, "Helvetica", 10, max_width)


def generate_pdf(data, filename='output.pdf'):
    c = canvas.Canvas(filename, pagesize=letter)
    global page_width, page_height, left_margin, right_margin, top_margin, bottom_margin
    page_width, page_height = letter

    left_margin = 40
    right_margin = 40
    top_margin = 60
    bottom_margin = 40

    c.setFont("Helvetica-Bold", 20)
    c.drawCentredString(page_width / 2.0, page_height - 40, "Nikles Data Report")

    y = page_height - top_margin

    # About section
    c.setFont("Helvetica-Bold", 18)
    c.drawString(left_margin, y, "About")
    y -= 30

    about_sections = ['Nikles in short', 'History', 'Philosophy']
    for section in about_sections:
        if section == 'History':
            y -= 20
            c.setFont("Helvetica-Bold", 14)
            c.drawString(left_margin, y, section)
            y -= 20
            for history_title, history_content in data['About'][section].items():
                y = draw_text_block(c, history_title, history_content['description'], y, 12, 10)
        else:
            y = draw_text_block(c, section, data['About'][section], y, 14, 10)

    # Technologies section
    c.setFont("Helvetica-Bold", 18)
    c.drawString(left_margin, y, "Technologies")
    y -= 30

    for key, value in data['Technologies'].items():
        y = draw_text_block(c, key, value, y, 14, 10)

    # Luxury Finishes section
    c.setFont("Helvetica-Bold", 18)
    c.drawString(left_margin, y, "Luxury Finishes")
    y -= 30

    luxury_sections = ['Info', 'Technology', 'Polished Items']
    for section in luxury_sections:
        if section == 'Polished Items':
            y -= 20
            c.setFont("Helvetica-Bold", 14)
            c.drawString(left_margin, y, section)
            y -= 20
            for item_title, item_desc in data['Luxury Finishes'][section].items():
                y = draw_text_block(c, item_title, item_desc, y, 12, 10)
        else:
            y = draw_text_block(c, section, data['Luxury Finishes'][section], y, 14, 10)
    
    # news section 
    c.setFont("Helvetica-Bold", 18)
    c.drawString(40, y, "News")
    y -= 30

    for title, info in data['News'].items():
        news_content = f"Author: {info['author']}\nCategory: {info['category']}\nPosted Time: {info['posted_time']}\nContent: {info['content']}"
        y = draw_text_block(c, title, news_content, y, 14, 10)

    c.save()

In [31]:
all_data = scrape_all()
# generate_pdf(all_data, '../data/nikles_data_report.pdf')